# QMUL Careers Team Python Script
### A detailed guide on how to run this programme.



Each text box describe what the code does in the cell below it.

The first cell imports some of the software that the programme needs, if you click the small play button above, or press Shift and Enter, it will run the cell.

In [11]:
import pandas as pd
import openpyxl
import numpy as np
import os
import ipywidgets as widgets

We need the excel workbook that the programme will use, and within that the excel spreadsheet with the student information. Please name the spreadsheet with the application information: "data" and save it on your computer.

The video below shows how to add this to the programme.

In [12]:
from IPython.display import Video; from ipywidgets import interactive, IntSlider

vid = Video(filename="media/add_applications.mov",data="", width=800, height = 600)
display(vid)

# enter sorting priority

Next, point the programme at the excel sheet just uploaded.

In [13]:
def get_sheet():
    for file in os.listdir('.'):
        if file.endswith(".xlsx"):
            myfile = file
            print(f'{file} has been selected.')
        else:
            pass           
    try:
        return myfile
    except:
        print('No file has been uploaded')
    
filename = get_sheet()

groups_assigned.xlsx has been selected.


Read the spreadsheet in, and perform some data cleaning.

In [14]:
df = pd.read_excel(filename,"data")
for i in df.columns:
    df.rename(columns={i:i.lower()}, inplace=True)
    print(i)

ValueError: Excel file format cannot be determined, you must specify an engine manually.

Enter the fields that you would like to prioritise. Each field you add will be used to sort the groups.

To stop running the cell type <b>'exit'</b> in the input box.

In [15]:
def add_cols():
    cols_to_sort = []
    count = 2
    while True:
        col = input('Enter the column you would like to add to the sorting algorithm, if there are no more type exit: ')
        if col.lower() == 'exit':
            [print(f'You have exited, your sorting columns are {str(cols_to_sort)}.')]
            break
        elif col in cols_to_sort:
            print('This column has already been added, please add another column or type exit to stop the programme.')
        elif col in df.columns:
            cols_to_sort.append(col)
            print(f'{col} will be used in the sorting algorithm. It will be priority number {count}')
            count+=1
        elif col not in df.columns:
            print(f'{col} is not a valid column. Please enter a valid column, or type exit to stop the programme.')
    return cols_to_sort
    
for_sorting = add_cols()

 is not a valid column. Please enter a valid column, or type exit to stop the programme.
 is not a valid column. Please enter a valid column, or type exit to stop the programme.
You have exited, your sorting columns are [].


Any missing scores will be replaced with the average score of the dataset.

In [16]:
df['total'].fillna(df['total'].mean(), inplace=True)

Some exploratory data analysis in the next 2 cells

In [17]:
df['gender'].value_counts()

F    54
M    47
Name: gender, dtype: int64

In [18]:
df['year of study'].value_counts()

1    46
2    35
3    20
Name: year of study, dtype: int64

Take only the columns important to establishing the groups. 

In [19]:
group_df = df[[ 
'student number',
'year of study',
'gender',
'faculty',
'total',
'course',
'personality type']]

'email', 
'Course', 
'YearOfStudy',
'Gender', 
'Bursary holder', 
'Client meetings delivery', 
'Total score',
'Personality type'

In [20]:
group_df = group_df.assign(groupno='')



#### The Sorting Algorithm

In [21]:
def assign_group(df):
    df.sort_values(by=for_sorting, ascending=False, inplace=True)
    count = 1
    up = True
    for i,row in group_df.iterrows():

        if count <= round(len(df)/5,0) and up == True:
            df.at[i,'groupno'] = count
            count += 1

        elif count > round(len(df)/5,0) and up == True:
            df.at[i,'groupno'] = count-1
            count-=1
            up = False

        if count > 1 and up == False:
            df.at[i,'groupno'] = count
            count -= 1
            
        elif count == 1 and up == False:
            df.at[i,'groupno'] = count
            up = True
    
    return df

assigned = assign_group(group_df)

    

# Data Analysis

In [22]:
assigned

,student number,year of study,gender,faculty,total,course,personality type,groupno
0,190862204,3,M,Humanities and Social Sciences,12.0,LLB FT Law,Architect INTJ,1
1,190058003,2,M,Humanities and Social Sciences,9.5,BSc(Econ) FT Economics,Protagonist ENFJ,2
2,200463937,2,M,Humanities and Social Sciences,9.5,BSc FT Business with Law,Commander ENTJ,3
3,210927883,1,M,Science and Engineering,10.0,BEng FT Materials Science and Engineering,Logician INTP,4
4,210283239,1,F,Humanities and Social Sciences,10.0,BA FT Intellectual History,Logician INTP,5
...,...,...,...,...,...,...,...,...
96,210393918,1,M,Humanities and Social Sciences,11.5,LLB FT Law,Protagonist ENFJ,17
97,210732438,1,F,Science and Engineering,11.5,BSc FT Computer Science,Campaigner ENFP,18
98,210442397,1,M,Science and Engineering,6.5,BEng FT Aerospace Engineering with Industrial ...,Commander ENTJ,19
99,180441563,1,F,Science and Engineering,7.5,BSc FT Biomedical Sciences,Architect INTJ,20


In [23]:
assigned.groupby(['groupno'])[['total']].mean().round(2).min()

total    8.33
dtype: float64

In [24]:
assigned.pivot_table(index=['groupno'], columns=['year of study'], aggfunc='size', fill_value=0)


year of study,1,2,3
groupno,,,
1,3,0,2
2,2,2,1
3,3,1,1
4,2,2,1
5,2,3,0
6,2,3,0
7,2,2,1
8,2,2,1
9,3,0,2


In [25]:
assigned.pivot_table(index=['groupno'], columns=['gender'], aggfunc='size', fill_value=0)


gender,F,M
groupno,,
1,1,4
2,1,4
3,4,1
4,1,4
5,4,1
6,4,1
7,3,2
8,3,2
9,4,1


### Send the groups back into the original spreadsheet

In [26]:
subset = assigned[['student number','groupno']]

final = pd.merge(df,subset, how='inner', on='student number')



In [32]:
final.to_excel("grouped_student_applications.xlsx",sheet_name='Grouped Students') 

## Make sure you download your new spreadsheet, you can copy and paste it into your previous working document.